#### Fitting method for electronegative plasma
<hr/>

Control Variables : <font color = 'sky blue'> $n_e,n_+,n_-,T_e, T_+,T_-, m_+,m_-$ </font>


<font size="2">
    [1] <a href="https://aip.scitation.org/doi/10.1063/1.4802252"> [2013] Langmuir probe analysis of highly electronegative plasmas</a> <br>
    [2] <a href="https://doi.org/10.1063/1.4903328"> [2014] Langmuir probe analysis in electronegative plasmas</a> <br><br>
<hr/>

Number of control variables can be reduced by set $m_+,m_-$ and $n_+ =n_e+n_-$. We just need 5 variables for fitting<br>
Ion composition setup : $SF_5^+,SF_5^-$ and m=127 AMU<br>
Fitting error within 5%

<hr/>

Import Modules

In [61]:
import numpy as np
from setup import *

Probe Setup & Ion Mass

In [60]:
print("\033[94m\tProbe Setup\033[0m")
print("Probe Radius : \033[31m{}\033[0m [mm]\nProbe Length : \033[31m{}\033[0m [mm]\n".format(rp*1e3,lp*1e3))
print("\033[94m\tIon Mass\033[0m")
print("Positive : \033[31m{}\033[0m [amu]\nNegative : \033[31m{}\033[0m [amu]".format(Mp/1.67e-27,Mn/1.67e-27))

	Probe Setup
Probe Radius : 0.025 [mm]
Probe Length : 6.0 [mm]

	Ion Mass
Positive : 127.0 [amu]
Negative : 127.0 [amu]


Fitting Method